In [1]:
import sqlite3
from sqlite3 import Error

def create_connection():
    conn = None
    try:
        # This creates a SQLite database in memory. To persist the data, replace ':memory:' with 'database_name.db'
        conn = sqlite3.connect(':memory:')
        print(f'successful SQLite connection with id {id(conn)}')
    except Error as e:
        print(e)
    return conn

def close_connection(conn):
    conn.close()
    print(f'connection closed for id {id(conn)}')




In [2]:
def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

def insert_data(conn, table, data):
    with conn:
        c = conn.cursor()
        placeholders = ', '.join('?' * len(data[0]))  # Create placeholders based on data length
        c.executemany(f'INSERT INTO {table} VALUES ({placeholders})', data)

def select_all_data(conn, table):
    with conn:
        c = conn.cursor()
        c.execute(f"SELECT * FROM {table}")
        rows = c.fetchall()
        for row in rows:
            print(row)



In [5]:
def test_referential_integrity(conn):
    c = conn.cursor()
    # Trying to insert a record into 'courses' that has a 'Student_Number' or 'Course_Name' 
    # that doesn't exist in the 'students' or 'course_details' table, respectively.
    try:
        c.execute("INSERT INTO courses VALUES (9999, 'Nonexistent Course', 80)")
        conn.commit()
    except sqlite3.IntegrityError as e:
        print("IntegrityError:", e)



In [4]:
def main():
    database = r":memory:" # In-memory database. Replace with 'database_name.db' to persist data

    students_table = """ CREATE TABLE IF NOT EXISTS students (
                            Student_Number integer PRIMARY KEY,
                            Student_Name text NOT NULL,
                            Support text,
                            Date_of_Birth text
                        ); """

    course_details_table = """CREATE TABLE IF NOT EXISTS course_details (
                                Course_Name text PRIMARY KEY,
                                Exam_Boards text NOT NULL,
                                Teacher_Name text NOT NULL
                            );"""

    courses_table = """CREATE TABLE IF NOT EXISTS courses (
                        Student_Number integer NOT NULL,
                        Course_Name text NOT NULL,
                        Exam_Score integer,
                        FOREIGN KEY (Student_Number) REFERENCES students (Student_Number),
                        FOREIGN KEY (Course_Name) REFERENCES course_details (Course_Name)
                    );"""

    conn = create_connection()

    if conn is not None:
        # create students table
        create_table(conn, students_table)
        students_data = [
            (1001, 'Bob Baker', 'No', '2001-08-25'),
            (1002, 'Sally Davies', 'Yes', '1999-10-02'),
            (1003, 'Mark Hanmill', 'No', '1995-06-05'),
            (1004, 'Anas Ali', 'No', '1980-08-03'),
            (1005, 'Cheuk Yin', 'Yes', '2002-05-01')
        ]
        insert_data(conn, 'students', students_data)
        select_all_data(conn, 'students')  # view the data

        # create course_details table
        create_table(conn, course_details_table)
        course_details_data = [
            ('Computer Science', 'BCS', 'Mr Jones'),
            ('Maths', 'AQA', 'Ms Parker')
        ]
        insert_data(conn, 'course_details', course_details_data)
        select_all_data(conn, 'course_details')  # view the data

        # create courses table
        create_table(conn, courses_table)
        courses_data = [
            (1001, 'Computer Science', 78),
            (1002, 'Maths', 55),
            (1003, 'Computer Science', 90),
            (1004, 'Maths', 70),
            (1005, 'Computer Science', 45)
        ]
        insert_data(conn, 'courses', courses_data)
        select_all_data(conn, 'courses')  # view the data

        # Test referential integrity
        test_referential_integrity(conn)

        close_connection(conn)
    else:
        print("Error! Cannot create the database connection.")

if __name__ == '__main__':
    main()


successful SQLite connection with id 3003085915968
(1001, 'Bob Baker', 'No', '2001-08-25')
(1002, 'Sally Davies', 'Yes', '1999-10-02')
(1003, 'Mark Hanmill', 'No', '1995-06-05')
(1004, 'Anas Ali', 'No', '1980-08-03')
(1005, 'Cheuk Yin', 'Yes', '2002-05-01')
('Computer Science', 'BCS', 'Mr Jones')
('Maths', 'AQA', 'Ms Parker')
(1001, 'Computer Science', 78)
(1002, 'Maths', 55)
(1003, 'Computer Science', 90)
(1004, 'Maths', 70)
(1005, 'Computer Science', 45)
connection closed for id 3003085915968


In [6]:

def test_referential_integrity(conn):
    c = conn.cursor()
    # Trying to insert a record into 'courses' that has a 'Student_Number' or 'Course_Name' 
    # that doesn't exist in the 'students' or 'course_details' table, respectively.
    try:
        c.execute("INSERT INTO courses VALUES (9999, 'Nonexistent Course', 80)")
        conn.commit()
    except sqlite3.IntegrityError as e:
        print("IntegrityError:", e)
